<a href="https://colab.research.google.com/github/shazzad-hasan/training-reproducable-deep-learning-models/blob/main/LeNet-5/lenet5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/shazzad-hasan/training-reproducable-deep-learning-models.git

Cloning into 'training-reproducable-deep-learning-models'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 76 (delta 30), reused 48 (delta 18), pack-reused 0
Unpacking objects: 100% (76/76), done.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/training-reproducable-deep-learning-models/LeNet-5

/content/training-reproducable-deep-learning-models/LeNet-5


In [5]:
!ls

helper_dataset.py   helper_plot.py   lenet5.ipynb
helper_evaluate.py  helper_train.py


In [6]:
# import required libraries
import torch
import torchvision

import numpy as np
import matplotlib.pyplot as plt

In [7]:
# import local helper functions
from helper_dataset import dataloader_mnist
from helper_train import train
from helper_evaluate import compute_loss_accuracy, set_all_seeds
from helper_plot import show_examples, show_sample_test_result

In [8]:
# check if cuda is available
train_on_gpu = torch.cuda.is_available()

if train_on_gpu:
  print("CUDA is available!")
else:
  print("CUDA is not available")

device = torch.device('cuda') if train_on_gpu else torch.device('cpu')

CUDA is available!


In [16]:
# settings
random_seed = 123
valid_size = 0.2
batch_size = 32
num_epochs = 10

set_all_seeds(random_seed)

In [10]:
from torchvision import transforms

data_transform  = transforms.Compose([transforms.Resize((32, 32)),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5,),(0.5,))]
    
)

train_loader, valid_loader, test_loader, classes = dataloader_mnist(
    batch_size = batch_size, 
    train_transform = data_transform,
    test_transform = data_transform,
    valid_size = valid_size
)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [11]:
print(classes)
num_class = len(classes)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [12]:
# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    print('Class labels of 10 examples:', labels[:10])
    break

Image batch dimensions: torch.Size([32, 1, 32, 32])
Image label dimensions: torch.Size([32])
Class labels of 10 examples: tensor([1, 5, 9, 1, 1, 9, 7, 5, 3, 9])


In [13]:
import torch.nn as nn

class LeNet5(nn.Module):

    def __init__(self, num_classes, grayscale=False):
        super().__init__()
        
        self.grayscale = grayscale
        self.num_classes = num_classes

        in_channels = 1 if self.grayscale else 3

        self.features = torch.nn.Sequential(nn.Conv2d(in_channels, 6, 5),
                                            nn.Tanh(),
                                            nn.MaxPool2d(2),
                                            nn.Conv2d(6, 16, 5),
                                            nn.Tanh(),
                                            nn.MaxPool2d(2))

        self.classifier = torch.nn.Sequential(nn.Linear(16*5*5, 120),
                                              nn.Tanh(),
                                              nn.Linear(120, 84),
                                              nn.Tanh(),
                                              nn.Linear(84, num_classes))
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        return logits

model = LeNet5(num_classes=10, grayscale=True)
# move model to the right device
model.to(device)

print(model)

LeNet5(
  (features): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Tanh()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=400, out_features=120, bias=True)
    (1): Tanh()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): Tanh()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)


In [14]:
import torch.optim as optim

# specify loss
criterion = nn.CrossEntropyLoss() # categorical cross-entropy loss

params = model.parameters()
optimizer = optim.SGD(params, lr=0.01)

lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, mode="min")

In [17]:
train_loss_list, valid_loss_list, train_acc_list, valid_acc_list = train(model, num_epochs, 
                                                                         train_loader, valid_loader,
                                                                         test_loader, optimizer, 
                                                                         criterion, device) 

Epoch: 1 	Train loss: 0.0904 	Validation loss: 0.0002 	Train accuracy : 97.7729	Validation accuracy: 97.6000
Validation loss decressed (inf --> 0.0002). Saving model ...
Epoch: 2 	Train loss: 0.0778 	Validation loss: 0.0008 	Train accuracy : 98.0750	Validation accuracy: 97.7083
Epoch: 3 	Train loss: 0.0686 	Validation loss: 0.0002 	Train accuracy : 98.2812	Validation accuracy: 97.7750
Validation loss decressed (0.0002 --> 0.0002). Saving model ...
Epoch: 4 	Train loss: 0.0617 	Validation loss: 0.0000 	Train accuracy : 98.4521	Validation accuracy: 97.9917
Validation loss decressed (0.0002 --> 0.0000). Saving model ...
Epoch: 5 	Train loss: 0.0562 	Validation loss: 0.0000 	Train accuracy : 98.5938	Validation accuracy: 98.0500
Validation loss decressed (0.0000 --> 0.0000). Saving model ...
Epoch: 6 	Train loss: 0.0519 	Validation loss: 0.0001 	Train accuracy : 98.6375	Validation accuracy: 98.1333
Epoch: 7 	Train loss: 0.0481 	Validation loss: 0.0000 	Train accuracy : 98.7958	Validation ac